In [111]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import ast

from fastDamerauLevenshtein import damerauLevenshtein
from opportunistic_planning import data, sequence
from imp import reload
from scipy.stats import friedmanchisquare, wilcoxon
reload(data)
reload(sequence)
pd.set_option('display.max_rows', 500)

In [40]:
# read data

df = pd.read_csv('all_task_environments_2021-01-26.csv', header=0)

In [44]:
# dict generation ~1 sec

%time distances_dict = data.generate_distances_dict(df)

CPU times: user 973 ms, sys: 12.1 ms, total: 985 ms
Wall time: 963 ms


In [88]:
# n=1: ~4min
# n=10: ~15min
# n=50: ~1h
# n=100: 2h 4min
%time results_new, results_sum = data.calculate_edit_distances_prequential(df, distances_dict=distances_dict, n=50)

/home/zauri/files/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/zauri/files/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


CPU times: user 1h 34min 24s, sys: 3.82 s, total: 1h 34min 27s
Wall time: 1h 34min 27s


In [89]:
results_new

,"c: 1.0; k: 0.0,0.1,1.1; x","c: 1.0; k: 0.0,0.1,1.1; y","c: 1.0; k: 0.0,0.1,1.1; z","c: 1.0; k: 0.0,0.1,1.1; xy","c: 1.0; k: 0.0,0.1,1.1; xz","c: 1.0; k: 0.0,0.1,1.1; yz","c: 1.0; k: 0.0,0.1,1.1; xyz","c: 1.1; k: 0.0,0.1,1.1; x","c: 1.1; k: 0.0,0.1,1.1; y","c: 1.1; k: 0.0,0.1,1.1; z",...,"c: 1.9; k: 0.8,0.9,1.9; x","c: 1.9; k: 0.8,0.9,1.9; y","c: 1.9; k: 0.8,0.9,1.9; z","c: 1.9; k: 0.8,0.9,1.9; xy","c: 1.9; k: 0.8,0.9,1.9; xz","c: 1.9; k: 0.8,0.9,1.9; yz","c: 1.9; k: 0.8,0.9,1.9; xyz",sequence,error,ID
0,0.291667,0.125000,0.291667,0.125000,0.291667,0.125000,0.125000,0.291667,0.125000,0.291667,...,0.291667,0.291667,0.291667,0.291667,0.291667,0.291667,0.291667,pocgkr,0.723,a1
1,0.225000,0.183333,0.225000,0.183333,0.225000,0.183333,0.183333,0.225000,0.183333,0.225000,...,0.225000,0.225000,0.100000,0.225000,0.225000,0.225000,0.225000,cgwpcfks,0.785,a3
2,0.142857,0.200000,0.000000,0.200000,0.142857,0.200000,0.200000,0.142857,0.200000,0.000000,...,0.000000,0.125000,0.200000,0.000000,0.000000,0.125000,0.000000,kfsfkspwg,0.806,a5
3,0.100000,0.142857,0.100000,0.142857,0.100000,0.142857,0.142857,0.100000,0.142857,0.100000,...,0.100000,0.000000,0.100000,0.100000,0.100000,0.000000,0.100000,pfkswkfsococg,0.862,a11
4,0.200000,0.166667,0.200000,0.166667,0.200000,0.166667,0.166667,0.200000,0.166667,0.200000,...,0.200000,0.000000,0.000000,0.200000,0.200000,0.000000,0.200000,wptgkfsoc,0.806,a13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175,0.245833,0.225000,0.225000,0.225000,0.162500,0.225000,0.100000,0.225000,0.225000,0.125000,...,0.266667,0.225000,0.125000,0.225000,0.100000,0.225000,0.100000,hhsdbg,0.723,v8
176,0.166667,0.125000,0.125000,0.000000,0.125000,0.125000,0.000000,0.125000,0.125000,0.125000,...,0.125000,0.125000,0.125000,0.000000,0.125000,0.125000,0.125000,hhsdgb,0.723,v9
177,0.125000,0.100000,0.100000,0.000000,0.000000,0.100000,0.000000,0.125000,0.000000,0.100000,...,0.125000,0.000000,0.100000,0.000000,0.000000,0.100000,0.000000,hhsgdb,0.723,v10
178,0.166667,0.000000,0.100000,0.000000,0.000000,0.100000,0.000000,0.166667,0.100000,0.100000,...,0.166667,0.000000,0.100000,0.000000,0.000000,0.100000,0.000000,hhsgdb,0.723,v11


In [103]:
#results_new.T.to_csv('results/results_all_n50_2021-01-27.csv', header=True, index=True)

In [47]:
lowest_sum, lowest_idx_sum, lowest_median_sum, list_mean_sum, results_mean_sum = data.get_lowest_error(results_sum)
lowest_sum, lowest_idx_sum, lowest_median_sum

(0.9511580348663674,
 Index(['c: 1.0; k: 0.3,0.4,1.8; xyz'], dtype='object'),
 0.9499999999999998)

In [90]:
lowest, lowest_idx, lowest_median, list_mean, results_mean = data.get_lowest_error(results_new)
lowest, lowest_idx, lowest_median

(0.13038085509995637,
 Index(['c: 1.2; k: 0.1,0.2,1.7; xyz'], dtype='object'),
 0.125)

In [91]:
lowest_idx[0]

'c: 1.2; k: 0.1,0.2,1.7; xyz'

In [84]:
def get_list_sorted(results):
    
    for col in list(results):
        if col != 'sequence' and col != 'error' and col != 'ID':
            results.loc['mean', col] = results[col].mean()
            results.loc['median', col] = results[col].median()
    lowest = sorted(results.loc['mean'])
    lowest_median = sorted(results.loc['median'])
    
    idx_list = []
                
    for i in lowest_median:
        idx = results.columns[(results.loc['median'] == i)]
        idx_list.append(idx)

    return lowest, results.columns[(results.loc['mean'] == lowest[0])], lowest_median, idx_list

In [112]:
low, idx, low_median, idx_list = get_list_sorted(results_new)
[(val, i) for val, i in zip(low_median[:1],idx_list[:1])]

[(0.125,
  Index(['c: 1.2; k: 0.0,0.1,1.1; xy', 'c: 1.3; k: 0.0,0.1,1.1; xy',
         'c: 1.8; k: 0.0,0.1,1.1; xy', 'c: 1.1; k: 0.1,0.2,1.1; xyz',
         'c: 1.0; k: 0.2,0.3,1.1; xy', 'c: 1.2; k: 0.2,0.3,1.1; xy',
         'c: 1.3; k: 0.2,0.3,1.1; xyz', 'c: 1.6; k: 0.2,0.3,1.1; xy',
         'c: 1.6; k: 0.2,0.3,1.1; xyz', 'c: 1.2; k: 0.3,0.4,1.1; xyz',
         ...
         'c: 1.9; k: 0.3,0.4,1.9; xy', 'c: 1.1; k: 0.4,0.5,1.9; xy',
         'c: 1.2; k: 0.4,0.5,1.9; xy', 'c: 1.2; k: 0.4,0.5,1.9; xyz',
         'c: 1.4; k: 0.4,0.5,1.9; xy', 'c: 1.5; k: 0.4,0.5,1.9; xy',
         'c: 1.7; k: 0.4,0.5,1.9; xyz', 'c: 1.1; k: 0.5,0.6,1.9; xyz',
         'c: 1.2; k: 0.5,0.6,1.9; xyz', 'c: 1.7; k: 0.5,0.6,1.9; xyz'],
        dtype='object', length=249))]

### Plot dimensions

In [100]:
# Define input for plots

c = [float(x[3:6]) for x in results_new.columns.tolist() if x != 'sequence' and x != 'ID' and x != 'error']
k = [float(x[11:14]) for x in results_new.columns.tolist() if x != 'sequence' and x != 'ID' and x != 'error']
dim = [x.strip() for x in results_new.columns[:-3].str.split(';').str[2]]
median_dims = [x for x in results_mean.loc['mean'].tolist() if pd.notnull(x)]

In [101]:
%matplotlib qt

#cm = ['red','blue','green','magenta','cyan','orange','grey']
cm = ['#e74c3c', '#0064a3', '#70b85d', '#287d78', '#54d0ff', '#f1e664', '#fd8f00']
dim_num = [0 if x=='x' else 1 if x=='y' else 2 if x=='z' else 3 if x=='xy' else 4 if x=='xz' else 5 if x=='yz' else 6 for x in dim]

cmap = matplotlib.colors.ListedColormap(cm)

ticks = ['x', 'y', 'z', 'xy', 'xz', 'yz', 'xyz']
norm = matplotlib.colors.BoundaryNorm(ticks, cmap.N)

# create figure, 3d grid, set background to white
fig2 = plt.figure(figsize=(12,8))
ax2 = fig2.add_subplot(111, projection='3d')
ax2.w_xaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_yaxis.set_pane_color((1.0,1.0,1.0,1.0))
ax2.w_zaxis.set_pane_color((1.0,1.0,1.0,1.0))

# create plot w/ median edit distance
img = ax2.scatter(c, k, median_dims, alpha=0.5, s=38, c=dim_num, cmap=cmap)

# plot horizontal plane for baseline
xx, yy = np.meshgrid(np.linspace(1,1.9), np.linspace(0,0.9))
#yy = np.meshgrid(range(2), range(2))
zz = xx * 0 + np.mean(rnn)
ax2.plot_surface(xx, yy, zz, alpha=0.5)

# set labels
ax2.set_ylabel('parameter k', fontsize=20, labelpad=10)
ax2.set_xlabel('parameter c', fontsize=20, labelpad=10)
ax2.set_zlabel('normalized edit distance', fontsize=20, labelpad=7)
#plt.title('Average edit distance (1 step)', fontsize=24)

# create colorbar
cb = plt.colorbar(img, cax = fig2.add_axes([0.82,0.3,0.03,0.4]))
cb.set_alpha(0.8)
cb.draw_all()
cb.ax.set_yticklabels(ticks, fontsize=16)
plt.show()

#plt.gca().xaxis.set_major_locator(plt.NullLocator())
#plt.gca().yaxis.set_major_locator(plt.NullLocator())
#plt.savefig('figures/test.svg', format='svg', bbox_inches='tight', pad_inches = 0)

/home/zauri/files/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: FixedFormatter should only be used together with FixedLocator


In [51]:
cpt = [0.25, 0.16666666666666663, 0.13392857142857145, 0.10555555555555557, 0.15476190476190477, 0.26666666666666666, 0.25, 0.22499999999999998, 0.125, 0.1833333333333333, 0.1428571428571429, 0.1428571428571429, 0.11805555555555558, 0.25, 0.25, 0.25, 0.15476190476190477, 0.25, 0.11111111111111116, 0.09545454545454546, 0.13392857142857145, 0.19999999999999996, 0.11249999999999999, 0.09545454545454546, 0.19999999999999996, 0.16666666666666663, 0.11111111111111116, 0.1833333333333333, 0.22499999999999998, 0.26666666666666666, 0.1428571428571429, 0.10555555555555557, 0.14583333333333331, 0.16666666666666663, 0.22499999999999998, 0.1428571428571429, 0.14583333333333331, 0.19999999999999996, 0.14583333333333331, 0.25, 0.16666666666666663, 0.2916666666666667, 0.1428571428571429, 0.20833333333333331, 0.16666666666666663, 0.25, 0.26666666666666666, 0.15476190476190477, 0.19999999999999996, 0.16666666666666663, 0.25, 0.22499999999999998, 0.10555555555555557, 0.22499999999999998, 0.19999999999999996, 0.14583333333333331, 0.1833333333333333, 0.22499999999999998, 0.1428571428571429, 0.2916666666666667, 0.33333333333333337, 0.13392857142857145, 0.5, 0.22499999999999998, 0.19999999999999996, 0.26666666666666666, 0.10555555555555557, 0.13392857142857145, 0.19999999999999996, 0.15476190476190477, 0.19999999999999996, 0.25, 0.1428571428571429, 0.16666666666666663, 0.14583333333333331, 0.13392857142857145, 0.13392857142857145, 0.22499999999999998, 0.12698412698412703, 0.16666666666666663, 0.2916666666666667, 0.2916666666666667, 0.2916666666666667, 0.2916666666666667, 0.4166666666666667, 0.2916666666666667, 0.33333333333333337, 0.25, 0.4166666666666667, 0.4166666666666667, 0.2916666666666667, 0.25, 0.33333333333333337, 0.1833333333333333, 0.25, 0.1428571428571429, 0.2916666666666667, 0.5, 0.19999999999999996, 0.2916666666666667, 0.16666666666666663, 0.16666666666666663, 0.19999999999999996, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.19999999999999996, 0.16666666666666663, 0.16666666666666663, 0.19999999999999996, 0.16666666666666663, 0.19999999999999996, 0.19999999999999996, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.16666666666666663, 0.19999999999999996, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.19999999999999996, 0.16666666666666663, 0.19999999999999996, 0.19999999999999996, 0.16666666666666663, 0.19999999999999996, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.19999999999999996, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.22499999999999998, 0.20833333333333331, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996]

In [52]:
rnn = [0.19999999999999996, 0.16666666666666663, 0.13392857142857145, 0.10101010101010105, 0.13392857142857145, 0.09999999999999998, 0.0, 0.125, 0.125, 0.11805555555555558, 0.09999999999999998, 0.1428571428571429, 0.09545454545454546, 0.19999999999999996, 0.25, 0.19999999999999996, 0.10101010101010105, 0.0, 0.09999999999999998, 0.08712121212121215, 0.11805555555555558, 0.19999999999999996, 0.08712121212121215, 0.0839160839160839, 0.1428571428571429, 0.16666666666666663, 0.0, 0.15476190476190477, 0.22499999999999998, 0.09999999999999998, 0.1428571428571429, 0.10555555555555557, 0.15476190476190477, 0.16666666666666663, 0.09999999999999998, 0.125, 0.12698412698412703, 0.16666666666666663, 0.10555555555555557, 0.0, 0.25, 0.2916666666666667, 0.125, 0.20833333333333331, 0.11111111111111116, 0.33333333333333337, 0.26666666666666666, 0.13392857142857145, 0.16666666666666663, 0.16666666666666663, 0.19999999999999996, 0.26666666666666666, 0.10101010101010105, 0.0, 0.16666666666666663, 0.1388888888888889, 0.0, 0.15476190476190477, 0.09999999999999998, 0.2916666666666667, 0.33333333333333337, 0.08012820512820512, 0.5, 0.22499999999999998, 0.0, 0.22499999999999998, 0.08712121212121215, 0.11805555555555558, 0.1428571428571429, 0.1833333333333333, 0.125, 0.33333333333333337, 0.125, 0.1428571428571429, 0.11805555555555558, 0.08012820512820512, 0.13392857142857145, 0.22499999999999998, 0.0, 0.1428571428571429, 0.09999999999999998, 0.22499999999999998, 0.09999999999999998, 0.2916666666666667, 0.4166666666666667, 0.09999999999999998, 0.33333333333333337, 0.25, 0.4166666666666667, 0.4166666666666667, 0.2916666666666667, 0.25, 0.33333333333333337, 0.1833333333333333, 0.25, 0.1428571428571429, 0.2916666666666667, 0.5, 0.25, 0.2916666666666667, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.19999999999999996, 0.1428571428571429, 0.0, 0.19999999999999996, 0.0, 0.125, 0.125, 0.19999999999999996, 0.16666666666666663, 0.0, 0.0, 0.1428571428571429, 0.16666666666666663, 0.16666666666666663, 0.19999999999999996, 0.1428571428571429, 0.1428571428571429, 0.16666666666666663, 0.16666666666666663, 0.0, 0.16666666666666663, 0.19999999999999996, 0.16666666666666663, 0.19999999999999996, 0.16666666666666663, 0.19999999999999996, 0.125, 0.16666666666666663, 0.16666666666666663, 0.16666666666666663, 0.19999999999999996, 0.1428571428571429, 0.19999999999999996, 0.125, 0.19999999999999996, 0.125, 0.0, 0.16666666666666663, 0.1428571428571429, 0.0, 0.19999999999999996, 0.16666666666666663, 0.19999999999999996, 0.0, 0.16666666666666663, 0.1428571428571429, 0.125, 0.16666666666666663, 0.1428571428571429, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16666666666666663, 0.16666666666666663, 0.0, 0.0, 0.0, 0.16666666666666663, 0.0, 0.16666666666666663, 0.16666666666666663, 0.1428571428571429, 0.2916666666666667, 0.08333333333333331, 0.25, 0.0, 0.25, 0.19999999999999996, 0.25, 0.0, 0.25, 0.19999999999999996, 0.0, 0.25]

In [149]:
len(cpt), len(rnn)

(180, 180)

In [114]:
results_new[lowest_idx[0]].loc[results_new[lowest_idx[0]] > 0.149].shape

(76,)

### Plot best model with baselines

In [102]:
#%matplotlib inline
%matplotlib qt

IDs = df['ID']

error = df['error']

seqs= results_new['sequence'][:-2].values

res = results_new[lowest_idx[0]][:-2].values
median = [np.nanmedian(res)] * len(res)

x = [x for x in range (0,len(seqs))]

# plot CPT baseline
plt.scatter(x, cpt, marker='o', s=20, c='dodgerblue', alpha=0.3, 
            label=str('CPT baseline median: ') + str(round(np.median(cpt),3)))
plt.plot(x, [np.median(cpt)] * len(x), '-', c='dodgerblue', alpha=0.9)
#plt.plot((x_original,x_original),(res_original,distances), '--', c='darkgreen', alpha=0.6)
plt.fill_between(x, cpt, alpha=0.3, color='dodgerblue')


# plot RNN baseline
plt.scatter(x, rnn, marker='o', s=20, c='limegreen', alpha=0.5, 
            label=str('RNN baseline median: ') + str(round(np.median(rnn),3)))
plt.plot(x, [np.median(rnn)] * len(x), '-', c='green', alpha=0.9)
#plt.plot((x_original,x_original),(res_original,distances), '--', c='darkgreen', alpha=0.6)
plt.fill_between(x, rnn, alpha=0.3, color='limegreen')

# plot random baseline
#plt.scatter(x, error, c='red', s=20, marker='o', alpha=0.3, 
#            label=str('guessing baseline median: ') + str(round(np.nanmedian(results_new['error']),3)))
#plt.plot(x, [np.median(error)] * len(x), c='red', alpha=0.8)#, 
         #label=str('guessing baseline avg.: ' + str(round(np.mean(results_original['error']),2))))
#plt.fill_between(x, error, [1.0 * len(error)], color='red', alpha=0.2)

# plot scatter + lines for simulations
plt.scatter(x, res, marker='o', s=20, c='darkviolet', alpha=0.8, 
            label=str('model-generated median: ') + str(round(lowest_median,3)))
#plt.plot(x, res, c='blue', alpha=0.6)
plt.plot(x, median, c='darkviolet', alpha=0.95)



plt.xticks(x, labels=IDs, rotation=90, fontsize=5)
#plt.xticklabels(IDs, rotation=90, fontsize=6)

plt.ylabel('normalized edit distance', fontsize=22)
plt.xlabel('sequence', fontsize=22)
plt.ylim(0.0, 0.51)
plt.title('best model (all), ' + str(lowest_idx[0]), fontsize=26)

plt.legend(fontsize=20, framealpha=0.8, loc='upper right', markerscale=2.5)

#plt.savefig('plot_median_editdist_individualerrors_diff.png', bbox_inches='tight')
plt.show()

### Prep data for stat analysis

In [96]:
# Compare avg edit distance for xy and xyz
list_xy = []
list_xyz = []

for col in results_mean:
    if col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'xy':
        list_xy.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'xyz':
        list_xyz.append(results_mean.at['mean',col])
        
avg_xy = np.mean(list_xy)
med_xy = np.median(list_xy)
std_xy = np.std(list_xy)
print('Average xy: ' + str(avg_xy)  + ', stdev: ' + str(std_xy) + ', median: ' + str(med_xy))
        
avg_xyz = np.mean(list_xyz)
med_xyz = np.median(list_xyz)
std_xyz = np.std(list_xyz)
print('Average xyz: ' + str(avg_xyz) + ', stdev: ' + str(std_xyz) + ', median: ' + str(med_xyz))

Average xy: 0.14312470045129033, stdev: 0.00865354436835479, median: 0.13852185776752732
Average xyz: 0.14298582294479512, stdev: 0.009043192742680442, median: 0.13811518126241923


In [97]:
# Compare avg edit distances for x, y, z
list_x = []
list_y = []
list_z = []
list_xz = []
list_yz = []

for col in results_mean:
    if col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'x':
        list_x.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'y':
        list_y.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'z':
        list_z.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'xz':
        list_xz.append(results_mean.at['mean',col])
    elif col != 'sequence' and col != 'ID' and col != 'error' and col.split(';')[2].strip() == 'yz':
        list_yz.append(results_mean.at['mean',col])

avg_xz = np.mean(list_xz)
avg_yz = np.mean(list_yz)
        
avg_x = np.mean(list_x)
lowest_x = min(list_x)
print('Average x: ' + str(avg_x) + ', min: ' + str(lowest_x)) 
        
avg_y = np.mean(list_y)
lowest_y = min(list_y)
print('Average y: ' + str(avg_y) + ', min: ' + str(lowest_y)) 
        
avg_z = np.mean(list_z)
lowest_z = min(list_z)
print('Average z: ' + str(avg_z) + ', min: ' + str(lowest_z)) 

print('Average xz: ' + str(avg_xz)) 
print('Average yz: ' + str(avg_yz)) 

Average x: 0.16667166265688388, min: 0.15707494228444624
Average y: 0.17176083194784644, min: 0.15944567396814585
Average z: 0.18307489005693356, min: 0.17189678688976445
Average xz: 0.15888312930596032
Average yz: 0.17127203706825533


In [98]:
# Compare all error predictions for dimensions
stat, p = friedmanchisquare(list_x,list_y,list_z,list_xy,list_xz,list_yz,list_xyz)
print('Friedman: stat = %.3f, p = % 10.3E' % (stat, p))

Friedman: stat = 3940.950, p =  0.000E+00


In [99]:
# 2D vs 3D median
stat, p = wilcoxon(list_xy, list_xyz, zero_method='wilcox')
print('Wilcoxon: W = %.3f, p = %.5f' % (stat, p))

Wilcoxon: W = 152971.000, p = 0.09104
